# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 5
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr5.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr5.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 306 306

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000073578,ENSG00000180104,ENSG00000066230,ENSG00000112877,ENSG00000113504,ENSG00000174358,ENSG00000164362,ENSG00000049656,ENSG00000153395,ENSG00000171421
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000073578,1.000000000,-0.05612548,0.00574107,-0.00739731,-0.005595571,0.07901083,-0.062200336,0.08899457,-0.11784006,-0.106602702
ENSG00000180104,-0.056125480,1.00000000,-0.01198212,0.05724206,-0.018769286,-0.17536181,0.051252334,0.17546878,0.01845199,-0.082228548
ENSG00000066230,0.005741070,-0.01198212,1.00000000,-0.24549811,0.131906294,0.05823369,-0.010682731,0.17484437,0.12153711,-0.346419367
ENSG00000112877,-0.007397310,0.05724206,-0.24549811,1.00000000,0.015312475,-0.15265107,0.027053136,0.05707100,0.12739757,0.026691514
ENSG00000113504,-0.005595571,-0.01876929,0.13190629,0.01531248,1.000000000,0.02573308,-0.009157251,0.00701811,0.03856403,0.069161785
ENSG00000174358,0.079010830,-0.17536181,0.05823369,-0.15265107,0.025733080,1.00000000,-0.041351802,-0.27964319,-0.01417308,-0.007056436


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 18

[1] -0.01336175 -0.02669122 -0.04567781 -0.05106241 -0.08514630 -0.15514271
 [7] -0.16526575 -0.16711387 -0.17623627 -0.22542424 -0.26783858 -0.43411740
[13] -0.45129093 -0.58947009 -0.65452124 -0.87875475 -0.93734720 -1.06010012

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 306 306

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000073578,ENSG00000180104,ENSG00000066230,ENSG00000112877,ENSG00000113504,ENSG00000174358,ENSG00000164362,ENSG00000049656,ENSG00000153395,ENSG00000171421
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000073578,1.000000000,-0.05612548,0.00574107,-0.00739731,-0.005595571,0.07901083,-0.062200336,0.08899457,-0.11784006,-0.106602702
ENSG00000180104,-0.056125480,1.00000000,-0.01198212,0.05724206,-0.018769286,-0.17536181,0.051252334,0.17546878,0.01845199,-0.082228548
ENSG00000066230,0.005741070,-0.01198212,1.00000000,-0.24549811,0.131906294,0.05823369,-0.010682731,0.17484437,0.12153711,-0.346419367
ENSG00000112877,-0.007397310,0.05724206,-0.24549811,1.00000000,0.015312475,-0.15265107,0.027053136,0.05707100,0.12739757,0.026691514
ENSG00000113504,-0.005595571,-0.01876929,0.13190629,0.01531248,1.000000000,0.02573308,-0.009157251,0.00701811,0.03856403,0.069161785
ENSG00000174358,0.079010830,-0.17536181,0.05823369,-0.15265107,0.025733080,1.00000000,-0.041351802,-0.27964319,-0.01417308,-0.007056436


,ENSG00000073578,ENSG00000180104,ENSG00000066230,ENSG00000112877,ENSG00000113504,ENSG00000174358,ENSG00000164362,ENSG00000049656,ENSG00000153395,ENSG00000171421
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000073578,1.00000000,-0.02385578,0.02586042,-0.04349128,-0.01609092,0.02876361,-0.063299546,0.03408823,-0.06983502,-0.03915242
ENSG00000180104,-0.02385578,1.00000000,0.01251901,0.05316903,-0.01981211,-0.12027728,-0.010225739,0.09022209,0.02813691,-0.02901164
ENSG00000066230,0.02586042,0.01251901,1.00000000,-0.10273345,0.04753210,0.02471952,-0.044812746,0.04009000,0.02585309,-0.13961061
ENSG00000112877,-0.04349128,0.05316903,-0.10273345,1.00000000,-0.01987190,-0.08991721,0.021639599,0.02951299,0.02900528,0.07745540
ENSG00000113504,-0.01609092,-0.01981211,0.04753210,-0.01987190,1.00000000,0.01719498,-0.006693077,0.01598707,0.02487355,0.01955773
ENSG00000174358,0.02876361,-0.12027728,0.02471952,-0.08991721,0.01719498,1.00000000,0.020803988,-0.12099929,-0.02425842,-0.01393593


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)